In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
import gc  
import os  
import time  
import math
import warnings 
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from functools import partial
from itertools import combinations  
from warnings import simplefilter 
import joblib  
import lightgbm as lgb  
import numpy as np  
import pandas as pd  
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from sklearn.model_selection import KFold, TimeSeriesSplit  
import polars as pl
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

diff_Number = 60
is_offline = False 
LGB = True
NN = False
is_train = True  
is_infer = True 
max_lookback = np.nan 
split_day = 435  
RUN_FOR_SUBMISSION=True



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [3]:
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [4]:
# strategy = tf.distribute.MirroredStrategy()
# print('Number of GPU devices: {}'.format(strategy.num_replicas_in_sync))

In [5]:
df=pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
test_target = df[df["date_id"]>=478].reset_index(drop=True)#==test
test_target = test_target["target"]
# df = df[~np.isnan(df["reference_price"])]
# # df = df.dropna(subset=["target"])
# df.reset_index(drop=True, inplace=True)
df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


In [6]:
df=df.set_index(["stock_id","date_id","seconds_in_bucket"]).sort_index()

In [7]:
df["lag_date1"] = df.groupby(["stock_id","seconds_in_bucket"])['target'].shift(1, fill_value=0)
df["lag_date2"] = df.groupby(["stock_id","seconds_in_bucket"])['target'].shift(2, fill_value=0)
df["lag_time1"] = df.groupby(["stock_id","date_id"])['target'].shift(1, fill_value=0)
df["lag_time2"] = df.groupby(["stock_id","date_id"])['target'].shift(2, fill_value=0)

In [8]:
df.head()

imbalance_size  imbalance_buy_sell_flag  \
stock_id date_id seconds_in_bucket                                            
0        0       0                      3180602.69                        1   
                 10                     1299772.70                        1   
                 20                     1299772.70                        1   
                 30                     1299772.70                        1   
                 40                     1218204.43                        1   

                                    reference_price  matched_size  far_price  \
stock_id date_id seconds_in_bucket                                             
0        0       0                         0.999812   13380276.64        NaN   
                 10                        1.000026   15261106.63        NaN   
                 20                        0.999919   15261106.63        NaN   
                 30                        1.000133   15261106.63        NaN   
                 40                        1.000455   15342674.90        NaN   

                                    near_price  bid_price  bid_size  \
stock_id date_id seconds_in_bucket                                    
0        0       0                         NaN   0.999812  60651.50   
                 10                        NaN   0.999812  13996.50   
                 20                        NaN   0.999812   4665.50   
                 30                        NaN   1.000026  55998.00   
                 40                        NaN   1.000241  14655.95   

                                    ask_price  ask_size       wap    target  \
stock_id date_id seconds_in_bucket                                            
0        0       0                   1.000026   8493.03  1.000000 -3.029704   
                 10                  1.000026  23519.16  0.999892  0.389814   
                 20                  0.999919  12131.60  0.999842  4.220009   
                 30                  1.000133  46203.30  1.000085  5.450249   
                 40                  1.000455  26610.45  1.000317  3.169775   

                                    time_id  row_id  lag_date1  lag_date2  \
stock_id date_id seconds_in_bucket                                          
0        0       0                        0   0_0_0        0.0        0.0   
                 10                       1  0_10_0        0.0        0.0   
                 20                       2  0_20_0        0.0        0.0   
                 30                       3  0_30_0        0.0        0.0   
                 40                       4  0_40_0        0.0        0.0   

                                    lag_time1  lag_time2  
stock_id date_id seconds_in_bucket                        
0        0       0                   0.000000   0.000000  
                 10                 -3.029704   0.000000  
                 20                  0.389814  -3.029704  
                 30                  4.220009   0.389814  
                 40                  5.450249   4.220009

In [9]:
df=df.reset_index()

In [10]:
def fill_none(df):
    df['imbalance_size'].fillna(df['imbalance_size'].median(), inplace=True)
    df['reference_price'].fillna(df['reference_price'].median(), inplace=True)
    df['far_price'].fillna(method='ffill', inplace=True)
    df['near_price'].fillna(method='bfill', inplace=True)
    df['far_price'].fillna(df['far_price'].mean(), inplace=True)
    df['near_price'].fillna(df['near_price'].mean(), inplace=True)
    df['matched_size'].fillna(df['matched_size'].mean(), inplace=True)
    df['bid_price'].fillna(df['bid_price'].mean(), inplace=True)
    df['ask_price'].fillna(df['ask_price'].mean(), inplace=True)
    df['wap'].fillna(df['wap'].mean(), inplace=True)
    df['target'].fillna(df['target'].mean(), inplace=True)
    return df
df=fill_none(df)

In [11]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,ask_price,ask_size,wap,target,time_id,row_id,lag_date1,lag_date2,lag_time1,lag_time2
0,0,0,0,3180602.69,1,0.999812,13380276.64,1.000792,1.000241,0.999812,...,1.000026,8493.03,1.000000,-3.029704,0,0_0_0,0.0,0.0,0.000000,0.000000
1,0,0,10,1299772.70,1,1.000026,15261106.63,1.000792,1.000241,0.999812,...,1.000026,23519.16,0.999892,0.389814,1,0_10_0,0.0,0.0,-3.029704,0.000000
2,0,0,20,1299772.70,1,0.999919,15261106.63,1.000792,1.000241,0.999812,...,0.999919,12131.60,0.999842,4.220009,2,0_20_0,0.0,0.0,0.389814,-3.029704
3,0,0,30,1299772.70,1,1.000133,15261106.63,1.000792,1.000241,1.000026,...,1.000133,46203.30,1.000085,5.450249,3,0_30_0,0.0,0.0,4.220009,0.389814
4,0,0,40,1218204.43,1,1.000455,15342674.90,1.000792,1.000241,1.000241,...,1.000455,26610.45,1.000317,3.169775,4,0_40_0,0.0,0.0,5.450249,4.220009


In [12]:
df=df.set_index(["stock_id","date_id","seconds_in_bucket"]).sort_index()
df=df.reset_index()

In [13]:
df

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,ask_price,ask_size,wap,target,time_id,row_id,lag_date1,lag_date2,lag_time1,lag_time2
0,0,0,0,3180602.69,1,0.999812,13380276.64,1.000792,1.000241,0.999812,...,1.000026,8493.03,1.000000,-3.029704,0,0_0_0,0.000000,0.000000,0.000000,0.000000
1,0,0,10,1299772.70,1,1.000026,15261106.63,1.000792,1.000241,0.999812,...,1.000026,23519.16,0.999892,0.389814,1,0_10_0,0.000000,0.000000,-3.029704,0.000000
2,0,0,20,1299772.70,1,0.999919,15261106.63,1.000792,1.000241,0.999812,...,0.999919,12131.60,0.999842,4.220009,2,0_20_0,0.000000,0.000000,0.389814,-3.029704
3,0,0,30,1299772.70,1,1.000133,15261106.63,1.000792,1.000241,1.000026,...,1.000133,46203.30,1.000085,5.450249,3,0_30_0,0.000000,0.000000,4.220009,0.389814
4,0,0,40,1218204.43,1,1.000455,15342674.90,1.000792,1.000241,1.000241,...,1.000455,26610.45,1.000317,3.169775,4,0_40_0,0.000000,0.000000,5.450249,4.220009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,199,480,500,2661783.46,-1,1.002764,23969215.23,1.000859,1.001494,1.002764,...,1.003082,425296.17,1.002835,-7.209778,26450,480_500_199,-4.659891,2.859831,-2.139807,4.299879
5237976,199,480,510,2658917.61,-1,1.002764,23972081.09,1.000859,1.001494,1.002764,...,1.003082,347268.87,1.002974,-9.750128,26451,480_510_199,-4.739761,3.679991,-7.209778,-2.139807
5237977,199,480,520,1352558.88,-1,1.001812,23978379.68,1.000859,1.001494,1.001812,...,1.002129,194630.52,1.001982,3.629923,26452,480_520_199,-5.440116,6.020069,-9.750128,-7.209778
5237978,199,480,530,1352558.88,-1,1.001812,23978379.68,1.000859,1.001494,1.001812,...,1.002129,214513.32,1.001993,4.760027,26453,480_530_199,-8.689761,10.600090,3.629923,-9.750128


In [14]:
# df=df.drop(["date_id","time_id","row_id","seconds_in_bucket"],axis=1)
# df=df.drop(["stock_id"],axis=1)

In [15]:
def reduce_mem_usage(df, verbose=0):
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)

    if verbose:
        print(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    return df

df = reduce_mem_usage(df, verbose=1)

Memory usage of dataframe is 839.22 MB
Memory usage after optimization is: 384.64 MB
Decreased by 54.17%


In [16]:
# df=df.drop(["row_id"],axis=1)

In [17]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,ask_price,ask_size,wap,target,time_id,row_id,lag_date1,lag_date2,lag_time1,lag_time2
0,0,0,0,3180602.750,1,0.999812,13380277.0,1.000792,1.000241,0.999812,...,1.000026,8493.030273,1.000000,-3.029704,0,0_0_0,0.0,0.0,0.000000,0.000000
1,0,0,10,1299772.750,1,1.000026,15261107.0,1.000792,1.000241,0.999812,...,1.000026,23519.160156,0.999892,0.389814,1,0_10_0,0.0,0.0,-3.029704,0.000000
2,0,0,20,1299772.750,1,0.999919,15261107.0,1.000792,1.000241,0.999812,...,0.999919,12131.599609,0.999842,4.220009,2,0_20_0,0.0,0.0,0.389814,-3.029704
3,0,0,30,1299772.750,1,1.000133,15261107.0,1.000792,1.000241,1.000026,...,1.000133,46203.300781,1.000085,5.450249,3,0_30_0,0.0,0.0,4.220009,0.389814
4,0,0,40,1218204.375,1,1.000455,15342675.0,1.000792,1.000241,1.000241,...,1.000455,26610.449219,1.000317,3.169775,4,0_40_0,0.0,0.0,5.450249,4.220009


In [18]:
if is_offline:
    df_train = df[df["date_id"] <= split_day]
    df_valid = df[df["date_id"] > split_day]
    df_train=df_train.drop(["time_id","row_id"],axis=1)
    df_valid=df_valid.drop(["time_id","row_id"],axis=1)
    print("Offline mode")
    print(f"train : {df_train.shape}, valid : {df_valid.shape}")
    
else:
    df_train = df
    df_train=df_train.drop(["time_id","row_id"],axis=1)
    print("Online mode")
df_train = reduce_mem_usage(df_train)

Online mode


In [19]:
# feature_columns = list(df_train.columns)

In [20]:
df_train.shape

(5237980, 19)

In [21]:
# train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
# testing2_x=df_train[train_indices]
# testing2_y= df_train['target'][train_indices]
# print(testing2_x.shape)

In [22]:
if LGB:
    import numpy as np
    import lightgbm as lgb
    
    lgb_params = {
        "objective": "mae",
        "n_estimators":6000,
        "num_leaves": 256,
        "subsample": 0.6,
        "colsample_bytree": 0.8,
#         "learning_rate": 0.00871,
        "learning_rate": 0.01,
        'max_depth': 11,
        "n_jobs": 4,
        "device": "gpu",
        "verbosity": -1,
        "importance_type": "gain",
#         "reg_alpha": 0.1,
        "reg_alpha": 0.2,
        "reg_lambda": 3.25
    }

    feature_columns = list(df_train.columns)
    feature_columns.remove("target")
    print(feature_columns)
    print(f"Features = {len(feature_columns)}")
    #print(f"Feature length = {len(feature_columns)}")

    num_folds = 10
    fold_size = 480 // num_folds
    gap = 5

    models = []
    models_cbt = []
    scores = []

    model_save_path = 'modelitos_para_despues' 
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)

    date_ids = df_train['date_id'].values

    for i in range(num_folds):
        start = i * fold_size
        end = start + fold_size
        if i < num_folds - 1:  # No need to purge after the last fold
            purged_start = end - 2
            purged_end = end + gap + 2
            train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
        else:
            train_indices = (date_ids >= start) & (date_ids < end)

        test_indices = (date_ids >= end) & (date_ids < end + fold_size)
        
        gc.collect()
        
        df_fold_train = df_train[train_indices]
        df_fold_train_target = df_train['target'][train_indices]
        df_fold_valid = df_train[test_indices]
        df_fold_valid_target = df_train['target'][test_indices]

        print(f"Fold {i+1} Model Training")
#         print(df_fold_train.shape, df_fold_train_target.shape, df_fold_valid.shape, df_fold_valid_target.shape)

        # Train a LightGBM model for the current fold
        lgb_model = lgb.LGBMRegressor(**lgb_params)
        lgb_model.fit(
            df_fold_train[feature_columns],
            df_fold_train_target,
            eval_set=[(df_fold_valid[feature_columns], df_fold_valid_target)],
            callbacks=[
                lgb.callback.early_stopping(stopping_rounds=100),
                lgb.callback.log_evaluation(period=100),
            ],
        )
        models.append(lgb_model)
        # Save the model to a file
        model_filename = os.path.join(model_save_path, f'doblez_{i+1}.txt')
        lgb_model.booster_.save_model(model_filename)
        print(f"Model for fold {i+1} saved to {model_filename}")

        # Evaluate model performance on the validation set
        #------------LGB--------------#
        fold_predictions = lgb_model.predict(df_fold_valid[feature_columns])
        fold_score = math.sqrt(mean_squared_error(fold_predictions, df_fold_valid_target))
        scores.append(fold_score)
        print(f":LGB Fold {i+1} RMSE: {fold_score}")
        #------------CBT--------------#
#         fold_predictions = cbt_model.predict(df_fold_valid[feature_columns])
#         fold_score_cbt = mean_absolute_error(fold_predictions, df_fold_valid_target)
#         scores.append(fold_score_cbt)
#         print(f"CBT Fold {i+1} MAE: {fold_score_cbt}")

        # Free up memory by deleting fold specific variables
        del df_fold_train, df_fold_train_target, df_fold_valid, df_fold_valid_target
        gc.collect()

    # Calculate the average best iteration from all regular folds
    average_best_iteration = int(np.mean([model.best_iteration_ for model in models]))

    # Update the lgb_params with the average best iteration
    final_model_params = lgb_params.copy()

    # final_model_params['n_estimators'] = average_best_iteration
    # print(f"Training final model with average best iteration: {average_best_iteration}")

    # Train the final model on the entire dataset
    num_model = 1

    for i in range(num_model):
        final_model = lgb.LGBMRegressor(**final_model_params)
        final_model.fit(
            df_train[feature_columns],
            df_train['target'],
            callbacks=[
                lgb.callback.log_evaluation(period=100),
            ],
        )
        # Append the final model to the list of models
        models.append(final_model)

['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'lag_date1', 'lag_date2', 'lag_time1', 'lag_time2']
Features = 18
Fold 1 Model Training
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 4.40413
[200]	valid_0's l1: 3.92922
[300]	valid_0's l1: 3.81416
[400]	valid_0's l1: 3.77167
[500]	valid_0's l1: 3.7477
[600]	valid_0's l1: 3.7301
[700]	valid_0's l1: 3.71498
[800]	valid_0's l1: 3.70236
[900]	valid_0's l1: 3.69151
[1000]	valid_0's l1: 3.68244
[1100]	valid_0's l1: 3.67509
[1200]	valid_0's l1: 3.66832
[1300]	valid_0's l1: 3.66216
[1400]	valid_0's l1: 3.65681
[1500]	valid_0's l1: 3.65186
[1600]	valid_0's l1: 3.64728
[1700]	valid_0's l1: 3.64337
[1800]	valid_0's l1: 3.63965
[1900]	valid_0's l1: 3.63593
[2000]	valid_0's l1: 3.63239
[2100]	valid_0's l1: 3.62928
[2200]	valid_0's l1: 3.62604
[2300]	valid_0

In [23]:
# print("ok")

In [24]:
# sample_prediction=pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv")
# revealed_targets=pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv")
# test=pd.read_csv("/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv")

In [25]:
# test.head()

In [26]:
# revealed_targets

In [27]:
# revealed_targets=revealed_targets.drop(["date_id","time_id","seconds_in_bucket"],axis=1)
# revealed_targets = revealed_targets.rename(columns={'revealed_date_id': 'date_id', 'revealed_target': 'target',"revealed_time_id": "time_id"})
# revealed_targets

In [28]:
# new_test=new_test=pd.merge(test,revealed_targets,on=["stock_id","date_id","time_id"])

In [29]:
# new_test=fill_none(new_test)
# new_test

In [30]:
# df_train

In [31]:
def generate(test,revealed_target):
    revealed_target=revealed_target.drop(["stock_id","date_id","seconds_in_bucket"],axis=1)
#     revealed_target= revealed_target[revealed_target['stock_id'] != 0
#     test=pd.merge(test,revealed_targets,on=["stock_id","date_id","seconds_in_bucket"],how="left")
    test = pd.merge(test,revealed_target, left_index=True, right_index=True, how='left')
#     test = pd.concat(["test","revealed_target"], ignore_index=True)
    test=test.set_index("row_id")
    test["lag_date1"] = test.groupby(["stock_id","seconds_in_bucket"])['revealed_target'].shift(0, fill_value=0)
    test["lag_date2"] = test.groupby(["stock_id","seconds_in_bucket"])['revealed_target'].shift(1, fill_value=0)
    test["lag_time1"] = test.groupby(["stock_id","date_id"])['revealed_target'].shift(1, fill_value=0)
    test["lag_time2"] = test.groupby(["stock_id","date_id"])['revealed_target'].shift(2, fill_value=0)
    test.rename(columns={"revealed_target":"target"}, inplace=True)
    test=fill_none(test)
    test=test.drop(["revealed_time_id","revealed_date_id","currently_scored","target"],axis=1)

    return test

In [32]:
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return w

In [33]:
if is_infer:
    import optiver2023
    optiver2023.make_env.func_dict['__called__'] = False
    env = optiver2023.make_env()
    iter_test = env.iter_test()
    counter = 0
    y_min, y_max = -64, 64
    qps, predictions = [], []
    pred = np.array([])
    save_dt = {}
    
    # Weights for each fold model
    if LGB:
        lgb_model_weights = weighted_average(models)
        #cbt_model_weights = weighted_average(models_cbt)
        
    start_time = time.time()
    for (test, revealed_targets, sample_prediction) in iter_test:
        now_time = time.time()
        check_sec = test["seconds_in_bucket"].unique()[0]
        cache = generate(test,revealed_targets)
#         for seconds_diff in range(10, diff_Number+10, 10):
#             if check_sec>=seconds_diff:
#                 cache = seconds_not0_calculate_Volatility(cache, save_dt["test_past"+str(0)], deal_col, seconds_diff)
#                 for past_sec in sorted(range(0, seconds_diff, 10), reverse=True)[:-1]:
#                     save_dt["test_past"+str(past_sec-10)] = save_dt["test_past"+str(past_sec)]
#                 save_dt["test_past"+str(seconds_diff-10)] = test
#             else:
#                 cache  = seconds_0_calculate_Volatility(cache, deal_col, seconds_diff)
#                 save_dt["test_past"+str(check_sec)] = test
        
        if counter > 0:
            cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
        feat = cache[-len(test):]
        feat= feat.fillna(0)
        feat=feat.astype(int)
        print(f"Feat Shape is: {feat.shape}")

        # Generate predictions for each model and calculate the weighted average
        if LGB:
            lgb_predictions = np.zeros(len(test))
            for model, weight in zip(models, lgb_model_weights):
                lgb_predictions += weight * model.predict(feat)

        predictions = lgb_predictions
        
        #Using mean predictions rather than zero sum
        final_predictions = predictions/len(models)# - np.mean(predictions)
        clipped_predictions = np.clip(final_predictions, y_min, y_max)
        pred = np.concatenate((pred, clipped_predictions))
        sample_prediction['target'] = clipped_predictions
        env.predict(sample_prediction)
        counter += 1
        qps.append(time.time() - now_time)
        if counter % 10 == 0:
            print(counter, 'qps:', np.mean(qps))
    time_cost = 1.146 * np.mean(qps)
    all_time = time.time()-start_time
    print(f"The code will take approximately {np.round(time_cost, 4)} hours to reason about")
    print(f"The code will take approximately {np.round(all_time, 4)} seconds to run")


This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
10 qps: 0.6912015676498413
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
20 qps: 0.6893331050872803
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
Feat Shape is: (200, 18)
30 qps: 0.7145682096481323
Feat Shape is: (200, 18)
Feat Shape is: (200